# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Load the data from [here](https://drive.google.com/file/d/1AmdvTSj5Fl86i4uHiOnZPFg8S8mAIreA/view?usp=sharing) and place it in the data folder

In [2]:
# Your code here:
data = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:
data.head(3)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:
data.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
df = data.pivot_table(index = 'Salary or Hourly', values = 'Name', aggfunc = 'count')
df

,Name
Salary or Hourly,
Hourly,8022
Salary,25161


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:
df2 = data.pivot_table(index = 'Department', values = 'Name', aggfunc = 'count')
df2

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


In [52]:
#paolo: good work!

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# Your code here:
from scipy import stats
from scipy.stats import ttest_1samp
from scipy.stats import norm
from scipy.stats import t

In [8]:
#H0 mean = $30 per hour
#H1 mean != $30 per hour
data_hourly = data[data["Hourly Rate"].notnull()]
std_dev = data_hourly['Hourly Rate'].std()
std_dev

12.11257268427681

In [35]:
sample = data_hourly[['Hourly Rate']].sample(50)
#paolo: why do you take a sample?
mean_hourly = sample.mean()
mean_hourly

Hourly Rate    34.437
dtype: float64

In [40]:
alpha = 0.05
mean_hourly
std_dev
assumed_mean = 30
marg_error = std_dev / len(sample)
#paolo: notice that you need to take the sqrt of
# len(sample) to calculate marge_error
z = (mean_hourly - assumed_mean) / marg_error
print(z)

Hourly Rate    20.388642
dtype: float64


In [11]:
p_value = stats.norm.sf(abs(z))*2
p_value

#P value is very small and much less than alpha (critical value) and therefore we can reject the null hypothesis and assume that the hourly wage is on average significantly different from $30 p/h

array([6.32872992e-08])

In [41]:
#paolo: good approach but numerical results are not correct
# check you result against builtin function
# st, p = ttest_1samp()

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [12]:
# Your code here:
#H0 mean = 86,000
#H1 mean > 86,000

data_police = data[data['Department']== 'POLICE']
police_sample = data_police['Annual Salary'].sample(50)
#paolo: why do you take a sample?
std_dev_police = police_sample.std()
std_dev_police

20375.30101779393

In [13]:
sample_mean = police_sample.mean()
sample_mean

89222.16

In [14]:
alpha = 0.05
sample_mean
std_dev_police
assumed_mean_police = 86000
marg_error_police = std_dev_police / len(police_sample)
z_police = (sample_mean - assumed_mean_police) / marg_error_police
z_police


7.907024286870813

In [42]:
p_value_police = stats.norm.sf(abs(z_police))
p_value_police

#P value is not greater than alpha (critical value) and therefore we can reject the null hypothesis and assume that the mean annual salary is larger than 86,000
#paolo: same comments as previous exercise

1.3180710541858724e-15

Using the `crosstab` function, find the department that has the most hourly workers. 

In [43]:
# Your code here:
split_hourly_salary = pd.crosstab(data['Department'], data['Salary or Hourly'])
split_hourly_salary['Hourly'].sort_values(ascending = False).head(1)


Department
STREETS & SAN    1862
Name: Hourly, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [44]:
#H0: wage = $35 
#H1: wage = less than $35

data_san = data[data['Department']== 'STREETS & SAN']
clean_data_san = data_san[data_san['Hourly Rate'].notnull()]['Hourly Rate'].head(3)
clean_data_san

7     35.60
21    21.43
24    35.60
Name: Hourly Rate, dtype: float64

In [45]:
# Your code here:
alpha = 0.05
san_mean = clean_data_san.mean()
mu = 35
sample_stdev = clean_data_san.std()
standard_error = sample_stdev / np.sqrt(len(clean_data_san))

In [46]:
t_value = (san_mean - mu) / standard_error
t_value

-0.8729710656316164

In [47]:
t, p = ttest_1samp(clean_data_san, mu)
print(t)
print(p)

#P Value is less than Alpha critical value so we can
#paolo: results not correct-try using builtin function to compare results
# I refer you to the solution

-0.8729710656316162
0.47473091348012264


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [48]:
# Your code here:
sample = data_hourly[['Hourly Rate']].sample(50)
std_dev = sample.std()
std_dev
#paolo: why do yo take a sample?

Hourly Rate    12.451931
dtype: float64

In [49]:
sample_mean = sample.mean()
sample_mean

Hourly Rate    34.7964
dtype: float64

In [50]:
z = stats.norm.interval(0.95)[1]
z

1.959963984540054

In [51]:
m_o_e = std_dev / np.sqrt(len(sample))
m_o_e

Hourly Rate    1.760969
dtype: float64

In [25]:
con_int = [sample_mean - (z * m_o_e) , sample_mean + (z * m_o_e)]
print('With 95% confidence, the true mean lies in the inverval:', con_int)
#paolo: good approach but num result not correct-check solutions

With 95% confidence, the true mean lies in the inverval: [Hourly Rate    30.01758
dtype: float64, Hourly Rate    36.04962
dtype: float64]


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [26]:
# Your code here:
police_sample
std_dev_police
sample_mean_police = police_sample.mean()
z = stats.norm.interval(0.95)[1]
m_o_e = std_dev_police / np.sqrt(len(police_sample))

con_int = [sample_mean_police - (z * m_o_e) , sample_mean_police + (z * m_o_e)]
print('With 95% confidence, the true mean lies in the inverval:', con_int)
#paolo: good approach but result not correct-you are not using
# the info about degrees of freedom
#paolo: I refer you to the solution

With 95% confidence, the true mean lies in the inverval: [83574.5184794327, 94869.8015205673]


In [53]:
#paolo check: I advice to review the part about hypt test

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [27]:
# Your code here:

